In [8]:
#!/usr/bin/env python
import os
import sys
if os.path.exists('/home/chieh/code/wuML'):
	sys.path.insert(0,'/home/chieh/code/wuML')
    
import wuml
import numpy as np
import torch
import wplotlib

wuml.jupyter_print('Here we use the advance objective')
wuml.jupyter_print(r'\max\limits_{\theta} \frac{1}{n}\sum_i W_i(y_i -f_{\theta}(x))^2 + \frac{\lambda}{n}\sum_i ReLu\left[ W_i(y_i -f_{\theta}(x)) \right]',latex=True)

<IPython.core.display.Math object>

In [9]:
data = wuml.wData(xpath='../../data/data.comp3.csv', batch_size=20, 
					label_type='continuous', label_column_name='finalga_best', 
					row_id_with_label=0, columns_to_ignore=['id'])

[X_train, X_test, y_train, y_test] = wuml.split_training_test(data, data_name='data.comp3', 
										data_path='../../data/', save_as='no saving',
										xdata_type="%.4f", ydata_type="%.4f", test_percentage=0.1)

λ = 0.8
X_train = wuml.center_and_scale(X_train)
weights = wuml.get_likelihood_weight(y_train)
weights = weights.get_data_as('Tensor')

def costFunction(x, y, ŷ, ind):
	relu = torch.nn.ReLU()

	W = torch.squeeze(weights[ind])
	n = len(ind)
	ŷ = torch.squeeze(ŷ)
	y = torch.squeeze(y)

	penalty = torch.sum(relu(W*(ŷ - y)))/n	# This will penalize predictions higher than true labels
	loss = torch.sum(W*((y - ŷ)**2))/n + λ*penalty
	return loss


λlist = [0, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8]
maxE_list = [500, 600, 700, 800, 900, 1000, 1200, 1400]

loopL = wuml.zip_with_joint_permutation(λlist,maxE_list)


In [10]:

for λ, mx in loopL:
    #bNet = wuml.basicNetwork(costFunction, X_train, networkStructure=[(200,'relu'),(200,'relu'),(200,'relu'),(1,'none')], max_epoch=700, learning_rate=0.001)
    bNet = wuml.basicNetwork(costFunction, X_train, networkStructure=[(10,'relu'),(10,'relu'),(1,'none')], max_epoch=700, learning_rate=0.001)
    bNet.train(print_status=False)

    Ŷ_train = bNet(X_train, output_type='ndarray')		#Takes Numpy array or Tensor as input and outputs a Tensor
    X_test = wuml.center_and_scale(X_test)
    Ŷ_test = bNet(X_test, output_type='ndarray')		#Takes Numpy array or Tensor as input and outputs a Tensor
    
    e = X_test.Y - np.squeeze(Ŷ_test)
    avgError = np.sum(np.absolute(e))/65
    print(λ, mx, avgError)
    


0 500 3.1658677991490385
0 600 3.8420739057291162


KeyboardInterrupt: 

In [ ]:
    #SR = wuml.summarize_regression_result(X_train.Y, Ŷ_train)
    #wuml.jupyter_print(SR.true_vs_predict(sort_based_on_label=True), display_all_rows=True)
#X_test = wuml.center_and_scale(X_test)
#Ŷ_test = bNet(X_test, output_type='ndarray')		#Takes Numpy array or Tensor as input and outputs a Tensor
#SR = wuml.summarize_regression_result(X_test.Y, Ŷ_test)
#wuml.jupyter_print(SR.true_vs_predict(sort_based_on_label=True),display_all_rows=True)

In [ ]:
#e = X_test.Y - np.squeeze(Ŷ_test)
#avgError = np.sum(np.absolute(e))/65
#print(avgError)